In [5]:
import json
import requests
import urllib
import copy

In [6]:
#https://pypi.org/project/gamma-viewer/
from gamma_viewer import GammaViewer
from IPython.display import display

In [7]:
def submit_to_ars(m,ars_url='https://ars.transltr.io/ars/api',arax_url='https://arax.ncats.io/legacy'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

def retrieve_ars_results(mid,ars_url='https://ars.transltr.io/ars/api'):
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    #print( j['status'] )
    results = {}
    for child in j['children']:
        #print(child['status'])
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                print(e)
                child['status'] = 'ARS Error'
        else:
            nresults = 0
        #print( child['status'], child['actor']['agent'],nresults )
    return results

In [8]:
def submit_to_devars(m):
    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

def retrieve_devars_results(m):
    return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def submit_to_ciars(m):
    return submit_to_ars(m,ars_url='https://ars.ci.transltr.io/ars/api',arax_url='https://arax.ncats.io')

def retrieve_ciars_results(m):
    return retrieve_ars_results(m,ars_url='https://ars.ci.transltr.io/ars/api')


In [9]:
def printjson(j):
    print(json.dumps(j,indent=4))

def print_json(j):
    printjson(j)

In [10]:
def parse(json):
    keylist = list(json.keys())
    for key in keylist:
        if key in KEYS:
            if (isinstance(json[KEYS.get(key)],dict)):
                parse(json[key])
        elif (isinstance(json[key],dict)):
            parse(json[key])

In [11]:
def name_lookup(text):
    url= f'https://name-resolution-sri.renci.org/lookup?string={text}&offset=0&limit=10'
    response = requests.post(url)
    printjson(response.json())

In [12]:
def one_hop_message(subject_category,object_category,predicate=None,subject_id = None, object_id = None):
    query_graph = {
    "nodes": {
        'a':{
            "category": subject_category
        },
        'b':{
            "category": object_category
        }
    },
    "edges": {
        'ab':{
            "subject": "a",
            "object": "b",
            #"predicate": predicate
        }
    }
    }
    if predicate is not None:
        query_graph['edges']['ab']['predicate'] = predicate
    if subject_id is not None:
        query_graph['nodes']['a']['id'] = subject_id
    if object_id is not None:
        query_graph['nodes']['b']['id'] = object_id
    message = {"message": {"query_graph": query_graph},'knowledge_graph':{"nodes": [], "edges": [],}, 'results':[]}
    return message

In [13]:
def keys_exist(element, *keys):
    if not isinstance(element, dict):
        raise AttributeError('keys_exists() expects dict as first argument.')
    if len(keys) == 0:
        raise AttributeError('keys_exists() expects at least two arguments, one given.')

    _element = element
    for key in keys:
        try:
            _element = _element[key]
            if _element is None:
                return False
        except KeyError:
            return False
    return True

def get_safe(element,*keys):
    '''
    :param element: JSON to be processed
    :param keys: list of keys in order to be traversed. e.g. "fields","data","message","results
    :return: the value of the terminal key if present or None if not
    '''
    if element is None:
        return None
    _element = element
    for key in keys:
        try:
            _element = _element[key]
            if _element is None:
                return None
            if key == keys[-1]:
                return _element
        except KeyError:
            return None
    return None

In [14]:
'''
This pulls approved drugs by pulling the MAX_FDA_APPROVAL_PHASE string from the description of the node
This is by no means a complete solution and I should touch base with Vlado about the implementation of 
FDA approval data from MolePro for a more complete solution.
'''

def get_approved(results):
    approved = []
    for r in results:
        nodes = get_safe(result[r],"message","knowledge_graph","nodes")
        if nodes is not None:
            for node in nodes:
                attributes = get_safe(nodes[node],"attributes")
                if attributes is not None:
                    for attribute in attributes:
                        if("original_attribute_name" in attribute):
                            if(attribute["original_attribute_name"]=="description"):
                                if("MAX_FDA_APPROVAL_PHASE: 4" in attribute["value"]):
                                    approved.append(node)
    return approved

In [15]:
def synonymize(ids, url="https://nodenormalization-sri.renci.org/1.1/get_normalized_nodes"):
        if( isinstance(ids,str) ):
            ids = [ids]
        params = {"curie":ids}
        url_curies = urllib.parse.urlencode(params, True)
        syn_url=url+"?"+url_curies
        syn_response=requests.get(syn_url)
        synonyms= syn_response.json()
        return synonyms
        
                    

In [16]:
def create_synonym_table(messages):
    synonyms={}
    for message in messages.values():
        nodes = get_safe(message,"message","knowledge_graph","nodes")
        if nodes is not None:
            sym = synonymize(nodes)
            if(len(synonyms)==0):
                synonyms= sym
            else:
                absent = []
                for node in nodes:
                    if(node in synonyms.items()):
                        continue
                    else:
                        absent.append(node)
                absent_syms = synonymize(absent)
                synonyms.update(absent_syms)
    return synonyms
                


In [17]:
def create_synonym_tables(messages):
    synonyms={}
    for entry in messages:
        actor = entry
        message = messages[actor]
        nodes = get_safe(message,"message","knowledge_graph","nodes")
        if nodes is not None:
            sym = synonymize(nodes)
            synonyms[actor]=sym
    return synonyms
                

In [18]:
def canonical_id_lists(synonym_tables):
    pks ={}
    for entry in synonym_tables:
        pks[entry]=[]
        for k,v in synonym_tables[entry].items():
            if v is None:
                pks[entry].append(k)
            else:
                canonical_id=v["id"]["identifier"]
                if (canonical_id not in pks[entry]):
                    pks[entry].append(canonical_id)
    return pks

In [19]:
def get_consensus_nodes(canonical_id_lists):
    consensus={}
    canon = copy.deepcopy(canonical_id_lists)
    my_keys= canon.keys()
    my_values=canon.values()
    for actor, id_list in canonical_id_lists.items():
        for actor2, id_list2 in canon.items():
            #Surely, there is a better way to do this with slices or something
            if(actor==actor2):
                continue
            else:
                s1 = set(id_list)
                s2 = set(id_list2)
                inter = (s1 & s2)
                if (inter):
                    for node in inter:
                        if node in consensus.keys():
                            consensus[node].add(actor)
                            consensus[node].add(actor2)
                        else:
                            consensus[node]={actor,actor2}      
        canon.pop(actor,None)
    return consensus
        

In [20]:
def create_query(disease_id,node_id):
    query={
        "message": {
            "query_graph": {
                "nodes": {
                    "n0": {
                        "ids": [
                            disease_id
                        ],
                        "categories": [
                            "biolink:Disease"
                        ]
                    },
                    "n1": {
                         "categories":[
                             "biolink:Gene"
                            ]
                    },
                    "n2": {
                        "ids":[node_id],
                        "categories":[
                            "biolink:ChemicalSubstance"
                           ]
                   }
                },
                "edges": {
                    "e0": {
                        "subject": "n0",
                        "object": "n1"

                    },
                    "e1": {
                        "subject": "n1",
                        "object": "n2"

                    }
                }
            }
        }
    }
    return query
    

In [21]:
def execute_batch(disease,drug_list):
    result_list=[]
    for drug in drug_list:
        query = create_query(disease,drug)
        result = submit_to_ciars(query)
        result_list.append(result)
    return result_list

In [29]:
def batch_consensus(result_list):
    node_list=[]
    synonyms={}
    chem=""
    for result in result_list:
        fetched_result=retrieve_ciars_results(result)
        for entry in fetched_result:
            actor = entry
            message = fetched_result[actor]
            nodes = get_safe(message,"message","knowledge_graph","nodes")
            #this chem get is hardcoded af.  Fix when possible
            chem = get_safe(message,"message","query_graph","nodes","n2","ids")[0]
            if nodes is not None:
                node_list.extend(nodes)
        sym=synonymize(node_list)
        synonyms[chem]=sym
        node_list=[]    
    canon = canonical_id_lists(synonyms)
    con=get_consensus_nodes(canon)
    return con

In [33]:
def get_name_from_id(curies,url="https://nodenormalization-sri.renci.org/1.1/get_normalized_nodes"):
        if( isinstance(curies,str) ):
            curies = [curies]
        params = {"curie":curies}
        url_curies = urllib.parse.urlencode(params, True)
        syn_url=url+"?"+url_curies
        syn_response=requests.get(syn_url)
        synonyms= syn_response.json()
        names={}
        for curie in curies:
            name = get_safe(synonyms,curie,"id","label")
            if name is not None:
                names[name]=curie
        return names
        

In [49]:
#This is some bubblegum and bailing wire shenanigans.  I apologize
def print_consensus(con):
    #sorting the dict by the length of the values (how many things if hit)
    #and then slicing off the first entry because that one is provided in the query
    for k in sorted(con,key=lambda k: len(con[k]),reverse=True)[1:]:
        key_name=get_name_from_id(k)
        value_names=get_name_from_id(con[k])
        print(key_name)
        print("Was found to interact with the following drugs")
        print(value_names)
        print("\n\n\n")


# APNEA

In [27]:
drug_list = [
    "PUBCHEM.COMPOUND:445354",
    "PUBCHEM.COMPOUND:3549980",
    "PUBCHEM.COMPOUND:3345",
    "PUBCHEM.COMPOUND:4058",
    "PUBCHEM.COMPOUND:4168",
    "PUBCHEM.COMPOUND:4595",
    "PUBCHEM.COMPOUND:4927",
    "PUBCHEM.COMPOUND:60815",
    "PUBCHEM.COMPOUND:5732",
    "PUBCHEM.COMPOUND:25096"
]

# Vitamin A
# Valproate
# Fentanyl
# Meperidine
# Metoclopramide
# Ondansetron
# Promethazine
# Remifentanil
# Zolpidem
# Hydroxyzine



'\nVitamin A\nValproate\nFentanyl\nMeperidine\nMetoclopramide\nOndansetron\nPromethazine\nRemifentanil\nZolpidem\nHydroxyzine\n\n'

In [28]:
my_result_list=execute_batch("MONDO:0005296",drug_list)

https://arax.ncats.io/?source=ARS&id=405773b7-13a7-4938-ac62-be594aa8b35e
https://arax.ncats.io/?source=ARS&id=6ea8496b-e7c5-47cd-b241-7082f4f97125
https://arax.ncats.io/?source=ARS&id=e9fca0ef-18b5-46ee-acec-df9a4c2933e9
https://arax.ncats.io/?source=ARS&id=42fe43fc-328d-4994-ba72-74aaeb507131
https://arax.ncats.io/?source=ARS&id=280c7a4b-15f5-4ec2-973c-373cb6fa3a4d
https://arax.ncats.io/?source=ARS&id=f27d7cfd-cef1-4cba-8b0d-b1cea4c2cf81
https://arax.ncats.io/?source=ARS&id=d910a7ba-197e-4869-85e2-aa0b5b7f761d
https://arax.ncats.io/?source=ARS&id=7d21d9a5-290e-4d28-9b46-ff34e6bbb2d2
https://arax.ncats.io/?source=ARS&id=2b2d080d-098f-4701-b98f-1cc7af1746f8
https://arax.ncats.io/?source=ARS&id=76516063-0782-4dbe-86e9-9a824048bbc2


In [97]:
# Note, I don't have anything in place to check that the above are done before moving on
# to finding a consensus.  So, probably give it a good 10-20 minutes for results to come back
# before moving on to finding the consensus from the result sets

In [31]:
con_list = batch_consensus(my_result_list)

In [50]:
con = con_list
print_consensus(con)

{'TY3H_HUMAN Tyrosine 3-monooxygenase (sprot)': 'UniProtKB:P07101'}
Was found to interact with the following drugs
{'Retinol': 'PUBCHEM.COMPOUND:445354', 'Valproate': 'PUBCHEM.COMPOUND:3549980'}




{'CP2D6_HUMAN Cytochrome P450 2D6 (sprot)': 'UniProtKB:P10635'}
Was found to interact with the following drugs
{'Retinol': 'PUBCHEM.COMPOUND:445354', 'Fentanyl': 'PUBCHEM.COMPOUND:3345'}






# INFANTILE CEREBRAL PALSY

In [93]:
drug_list = [
    "PUBCHEM.COMPOUND:5381",
    "PUBCHEM.COMPOUND:5284627",
    "PUBCHEM.COMPOUND:2118",
    "PUBCHEM.COMPOUND:2803",
    "PUBCHEM.COMPOUND:2477",
    "PUBCHEM.COMPOUND:17836",
    "PUBCHEM.COMPOUND:135398745",
    "PUBCHEM.COMPOUND:4168",
    "PUBCHEM.COMPOUND:5284604"
]

# Tazarotene
# Topiramate
# Alprazolam
# Clonidine
# Buspirone
# Lorazepam
# Olanzapine
# Metolopramide
# Oxymorphone



'\nTazarotene\nTopiramate\nAlprazolam\nClonidine\nBuspirone\nLorazepam\nOlanzapine\nMetolopramide\nOxymorphone\n\n'

In [94]:
icp_results = execute_batch("MONDO:0006497",drug_list)


https://arax.ncats.io/?source=ARS&id=36212cd3-0f73-4c6b-9380-6f389618874d
https://arax.ncats.io/?source=ARS&id=5eecfee4-a22e-4134-9a17-d250cfe27058
https://arax.ncats.io/?source=ARS&id=463f46df-3f9b-4a62-a527-23dce73527dd
https://arax.ncats.io/?source=ARS&id=869a3b70-be95-47f0-af6e-026d8d94b8c3
https://arax.ncats.io/?source=ARS&id=fb35bb5c-046a-4a64-a860-77d7ed157d01
https://arax.ncats.io/?source=ARS&id=37f6ddd4-57a6-42a6-b4ba-26267e33963d
https://arax.ncats.io/?source=ARS&id=a0853774-ac14-44c5-baeb-545f64f946fd
https://arax.ncats.io/?source=ARS&id=cf3257f8-c8e8-4cd8-a675-1649719a5a6c
https://arax.ncats.io/?source=ARS&id=75c7b6cc-04bd-42cf-81c7-156650b59e0a


In [ ]:
# Note, I don't have anything in place to check that the above are done before moving on
# to finding a consensus.  So, probably give it a good 10-20 minutes for results to come back
# before moving on to finding the consensus from the result sets

In [98]:
consensus = batch_consensus(icp_results)


In [99]:
print_consensus(consensus)

MONDO:0006497 {'PUBCHEM.COMPOUND:4168', 'PUBCHEM.COMPOUND:5284627', 'PUBCHEM.COMPOUND:5284604', 'PUBCHEM.COMPOUND:5381', 'PUBCHEM.COMPOUND:2477', 'PUBCHEM.COMPOUND:2803', 'PUBCHEM.COMPOUND:135398745', 'PUBCHEM.COMPOUND:2118'}
UniProtKB:Q08117 {'PUBCHEM.COMPOUND:5284627', 'PUBCHEM.COMPOUND:5284604', 'PUBCHEM.COMPOUND:2803', 'PUBCHEM.COMPOUND:5381'}
UniProtKB:P07101 {'PUBCHEM.COMPOUND:5284627', 'PUBCHEM.COMPOUND:2803', 'PUBCHEM.COMPOUND:135398745'}
UniProtKB:P01189 {'PUBCHEM.COMPOUND:5284627', 'PUBCHEM.COMPOUND:135398745'}
UniProtKB:P42574 {'PUBCHEM.COMPOUND:2803', 'PUBCHEM.COMPOUND:2477'}
UniProtKB:Q96P20 {'PUBCHEM.COMPOUND:4168', 'PUBCHEM.COMPOUND:2803'}
UniProtKB:Q9NSE2 {'PUBCHEM.COMPOUND:4168', 'PUBCHEM.COMPOUND:135398745'}
